In [17]:
import pandas as pd
import folium
from IPython.display import display

# Load data
file_path = 'EchoPulse_List.csv'  
data = pd.read_csv(file_path, on_bad_lines='skip')

# Ensure 'recordedAt' column is in datetime format
data['recordedAt'] = pd.to_datetime(data['recordedAt'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# Count moving and stop points
total_moving_points = data[data['activity'] == 'Moving'].shape[0]
total_stop_points = data[data['activity'] == 'Stop'].shape[0]

print(f"Total Number of Moving Points: {total_moving_points}")
print(f"Total Number of Stop Points: {total_stop_points}")

# Filter moving and stop data
moving_data = data[data['activity'] == 'Moving']
stop_data = data[data['activity'] == 'Stop']

# Create folium map centered at the first recorded location
if not data.empty:
    map_center = [data['latitude'].iloc[0], data['longitude'].iloc[0]]
    m = folium.Map(location=map_center, zoom_start=12)
    
    # Combine moving and stop data for sequential numbering
    combined_data = pd.concat([moving_data, stop_data]).sort_values(by='recordedAt').reset_index()
    
    # Assign numbers to each point
    for idx, row in combined_data.iterrows():
        color = '#023690' if row['activity'] == 'Moving' else '#FF5358'  # Light blue and light red
        folium.CircleMarker(
            [row['latitude'], row['longitude']],
            radius=12,  # Increased radius for better text fit
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9
        ).add_to(m)
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.DivIcon(html=f'<div style="width: 24px; height: 24px; border-radius: 50%; background-color: transparent; display: flex; align-items: center; justify-content: center; font-size: 10pt; font-weight: bold; color: white;">{idx + 1}</div>')
        ).add_to(m)
    
    display(m)
else:
    print("No location data found.")


Total Number of Moving Points: 257
Total Number of Stop Points: 26


In [19]:
import pandas as pd
import folium
from folium import plugins
from datetime import datetime
from IPython.display import display

# Load data 
file_path = 'EchoPulse_List.csv' 
data = pd.read_csv(file_path)

# Ensure 'recordedAt' column is in datetime format
data['recordedAt'] = pd.to_datetime(data['recordedAt'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# Extract required columns for animation
locations = [
    {
        "lat": row['latitude'], 
        "lon": row['longitude'], 
        "time": row['recordedAt'].strftime("%Y-%m-%dT%H:%M:%S.%fZ"),
        "activity": row['activity']
    }
    for _, row in data.iterrows()
]

# Step 1: Create the map centered at the first point
if locations:
    map_center = [locations[0]['lat'], locations[0]['lon']]
    m_time = folium.Map(location=map_center, zoom_start=12)

    # Step 2: Create a feature group for the real-time animation
    route_animation = plugins.TimestampedGeoJson(
        {
            "type": "FeatureCollection",
            "features": [
                {
                    "type": "Feature",
                    "geometry": {
                        "type": "Point",
                        "coordinates": [loc['lon'], loc['lat']],  # Longitude first, latitude second
                    },
                    "properties": {
                        "time": datetime.strptime(loc['time'], "%Y-%m-%dT%H:%M:%S.%fZ").isoformat(),
                        "popup": f"Time: {loc['time']}",
                        "icon": "circle",
                        "iconstyle": {
                            "fillColor": "#023690" if loc['activity'] == "Moving" else "#FF5358",  # Moving: Blue, Stopped: Red
                            "fillOpacity": 0.8,
                            "stroke": "true",
                            "radius": 10  # Increased size
                        },
                    }
                } for loc in locations
            ]
        },
        period="PT30S",  # One point per 30 seconds
        add_last_point=True,
        auto_play=True,  # Automatically start the animation
        loop=True,       # Loop the animation
        max_speed=1,     # Playback speed
        loop_button=True,
        date_options='YYYY-MM-DD HH:mm:ss',  # Format of the displayed date and time
        time_slider_drag_update=True        # Enable real-time updates via slider
    ).add_to(m_time)

    # Step 3: Display the map
    display(m_time)
else:
    print("No location data available for animation.")
